In [4]:
import os
import tempfile

import lightgbm as lgb
import optuna
import pandas as pd
from datasetsforecast.m4 import M4, M4Evaluation, M4Info
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from utilsforecast.plotting import plot_series

from mlforecast import MLForecast
from mlforecast.auto import (
    AutoLightGBM,
    AutoMLForecast,
    AutoModel,
    AutoRidge,
    ridge_space,
)
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean

In [5]:
def get_data(group, horizon):
    df, *_ = M4.load(directory='data', group=group)
    df['ds'] = df['ds'].astype('int')
    df['unique_id'] = df['unique_id'].astype('category')
    return df.groupby('unique_id').head(-horizon).copy()

group = 'Hourly'
horizon = M4Info[group].horizon
train = get_data(group, horizon)

2.35MiB [00:00, 79.6MiB/s]                 
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Hourly-train.csv, 2347115, bytes.
133kiB [00:00, 36.9MiB/s]                   
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Hourly-test.csv, 132820, bytes.
4.34MiB [00:00, 106MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded M4-info.csv, 4335598, bytes.
100%|██████████| 3.56M/3.56M [00:00<00:00, 34.6MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded submission-Naive2.zip, 3564691, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/m4/datasets/submission-Naive2.zip


In [6]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
auto_mlf = AutoMLForecast(
    models={'lgb': AutoLightGBM(), 'ridge': AutoRidge()},
    freq=1,
    season_length=24,
)
auto_mlf.fit(
    train,
    n_windows=2,
    h=horizon,
    num_samples=2,  # number of trials to run
)

AutoMLForecast(models={'lgb': AutoModel(model=LGBMRegressor), 'ridge': AutoModel(model=Ridge)})

In [7]:
preds = auto_mlf.predict(horizon)
preds.head()

,unique_id,ds,lgb,ridge
0,H1,701,680.534943,604.140123
1,H1,702,599.038307,523.364874
2,H1,703,572.808421,479.174481
3,H1,704,564.573783,444.540062
4,H1,705,543.046026,419.987657


In [8]:
def evaluate(df, group):
    results = []
    for model in df.columns.drop(['unique_id', 'ds']):
        model_res = M4Evaluation.evaluate(
            'data', group, df[model].to_numpy().reshape(-1, horizon)
        )
        model_res.index = [model]
        results.append(model_res)
    return pd.concat(results).T.round(2)

evaluate(preds, group)

,lgb,ridge
SMAPE,18.78,20.00
MASE,5.07,1.29
OWA,1.57,0.81


In [9]:
def my_lgb_config(trial: optuna.Trial):
    return {
        'learning_rate': 0.05,
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 2, 128, log=True),
        'objective': trial.suggest_categorical('objective', ['l1', 'l2', 'mape']),
    }

my_lgb = AutoModel(
    model=lgb.LGBMRegressor(),
    config=my_lgb_config,
)
auto_mlf = AutoMLForecast(
    models={'my_lgb': my_lgb},
    freq=1,
    season_length=24,
).fit(
    train,
    n_windows=2,
    h=horizon,
    num_samples=2,
)
preds = auto_mlf.predict(horizon)
evaluate(preds, group)

,my_lgb
SMAPE,18.67
MASE,4.79
OWA,1.51
